In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd

In [3]:
year = 2021
month = 2

# input_file = f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet'
input_file = f'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/nyc-tlc/fhv/fhv_tripdata_{year:04d}-{month:02d}.parquet'


In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [8]:
from typing import List
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename: str, features: List[str]):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [9]:
df = read_data(input_file, features=categorical)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [11]:
y_pred.mean()

16.191691679979066

In [17]:
df_prediction = pd.DataFrame()
df_prediction['ride_id'] = df['ride_id']
df_prediction['prediction'] = y_pred
df_prediction

,ride_id,prediction
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


In [26]:
import pathlib
import os

folder_path = pathlib.Path("output")
folder_path.mkdir(parents=True, exist_ok=True)

output_file = pathlib.Path(folder_path, f'fhv_tripdata_{year:04d}-{month:02d}.parquet')

df_prediction.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)